In [28]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import re
import numpy as np
from datetime import datetime

In [3]:
#importing the excel spreadsheet containing my police data as a csv. Also using .head() and .info() to take a look at all the rows in the dataset to prepare for cleaning. 

df = pd.read_csv(r"C:\Users\kesha\Documents\nss_da10\projects\police-response-capstone\data\Metro_Nashville_Police_Department_Incidents_20240217.csv")

In [4]:
df.head()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,7/1/2019 17:31,7/1/2019 18:17,2410 2410,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218)
1,20180290646_11,20180290646,D,DISPATCHED,A,CLEARED BY ARREST,Closed,3/31/2018 12:21,3/31/2018 19:41,5824 5824,...,NONE,1,False,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.713 36.042)
2,20200243837_11,20200243837,D,DISPATCHED,U,UNFOUNDED,Closed,4/6/2020 11:15,4/6/2020 13:11,3144 3144,...,NONE,1,False,G,GOVERNMENT,NaN,NaN,NaN,NaN,POINT (-86.759 36.232)
3,20170471747_11,20170471747,D,DISPATCHED,U,UNFOUNDED,Closed,5/29/2017 2:33,5/29/2017 4:27,1714 1714,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.907 36.086)
4,20171099391_21,20171099391,D,DISPATCHED,A,CLEARED BY ARREST,Closed,12/16/2017 18:30,12/16/2017 22:44,1101 1101,...,PERSONAL (HANDS),1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.586 36.178)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865175 entries, 0 to 865174
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Primary Key                  865175 non-null  object 
 1   Incident Number              865175 non-null  int64  
 2   Report Type                  865089 non-null  object 
 3   Report Type Description      805823 non-null  object 
 4   Incident Status Code         865155 non-null  object 
 5   Incident Status Description  865145 non-null  object 
 6   Investigation Status         865175 non-null  object 
 7   Incident Occurred            865175 non-null  object 
 8   Incident Reported            865175 non-null  object 
 9   Incident Location            859872 non-null  object 
 10  Latitude                     848238 non-null  float64
 11  Longitude                    848238 non-null  float64
 12  ZIP Code                     454148 non-null  float64
 13 

In [8]:
#pulled in the only columns I needed from the entire police inident sheet
officers = df[['Report Type Description','Investigation Status','Incident Occurred','Incident Reported','ZIP Code']]
officers


,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [14]:
mnpd_officers = officers.loc[officers['Report Type Description'] == "DISPATCHED"]
mnpd_officers



,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [16]:
npd_officers = mnpd_officers.loc[officers['Investigation Status'] == "Closed"]
npd_officers

,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0
865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0
865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0
865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0


In [17]:
npd_officers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362799 entries, 0 to 865170
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Report Type Description  362799 non-null  object 
 1   Investigation Status     362799 non-null  object 
 2   Incident Occurred        362799 non-null  object 
 3   Incident Reported        362799 non-null  object 
 4   ZIP Code                 362045 non-null  float64
dtypes: float64(1), object(4)
memory usage: 16.6+ MB


In [20]:
npd_officers['Incident Occurred'] = npd_officers['Incident Occurred'].astype("string")
npd_officers['Incident Reported'] = npd_officers['Incident Reported'].astype("string")

C:\Users\kesha\AppData\Local\Temp\ipykernel_5064\1760736431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npd_officers['Incident Occurred'] = npd_officers['Incident Occurred'].astype("string")
C:\Users\kesha\AppData\Local\Temp\ipykernel_5064\1760736431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npd_officers['Incident Reported'] = npd_officers['Incident Reported'].astype("string")


In [21]:
npd_officers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362799 entries, 0 to 865170
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Report Type Description  362799 non-null  object 
 1   Investigation Status     362799 non-null  object 
 2   Incident Occurred        362799 non-null  string 
 3   Incident Reported        362799 non-null  string 
 4   ZIP Code                 362045 non-null  float64
dtypes: float64(1), object(2), string(2)
memory usage: 16.6+ MB


In [30]:
npd_officers['Incident Occurred Time'] = npd_officers['Incident Occurred'].apply(lambda x: datetime.strptime(x, '%H:%M'))
npd_officers['Incident Reported Time'] = npd_officers['Incident Reported'].apply(lambda x: datetime.strptime(x, '%H:%M'))

ValueError: time data '7/1/2019 17:31' does not match format '%H:%M'

In [27]:
npd_officers['Incident Occurred Time'] = pd.to_datetime(npd_officers['Incident Occurred'], format='%H:%M:%S')
npd_officers['Incident Reported Time'] = pd.to_datetime(npd_officers['Incident Reported'], format='%H:%M:%S')

ValueError: time data "7/1/2019 17:31" doesn't match format "%H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [32]:
npd_officers['Incident Occurred Datetime'] = pd.to_datetime(npd_officers['Incident Occurred'])

C:\Users\kesha\AppData\Local\Temp\ipykernel_5064\2990356056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npd_officers['Incident Occurred Datetime'] = pd.to_datetime(npd_officers['Incident Occurred'])


In [34]:
npd_officers['Incident Reported Datetime'] = pd.to_datetime(npd_officers['Incident Reported'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 4240-06-05 11:16:00, at position 295059. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [35]:
npd_officers_clean = npd_officers

In [40]:
npd_officers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362799 entries, 0 to 865170
Data columns (total 6 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Report Type Description     362799 non-null  object        
 1   Investigation Status        362799 non-null  object        
 2   Incident Occurred           362799 non-null  string        
 3   Incident Reported           362799 non-null  string        
 4   ZIP Code                    362045 non-null  float64       
 5   Incident Occurred Datetime  362799 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2), string(2)
memory usage: 27.4+ MB


In [39]:
npd_officers_clean = npd_officers_clean.drop(["Incident Occured"],
                                             axis = 295058)

ValueError: No axis named 295058 for object type DataFrame

In [43]:
npd_officers_clean = npd_officers_clean.reset_index()

In [44]:
npd_officers_clean

,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime
0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00
1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00
2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00
3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00
4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00
...,...,...,...,...,...,...,...
362793,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00
362794,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00
362795,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00
362796,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00


In [45]:
npd_officers_clean = npd_officers_clean.drop(295058)

In [47]:
npd_officers_clean['Incident Reported Datetime'] = pd.to_datetime(npd_officers_clean['Incident Reported'])

In [49]:
npd_officers_clean

,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,Incident Reported Datetime
0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019-07-01 18:17:00
1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018-03-31 19:41:00
2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020-04-06 13:11:00
3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017-05-29 04:27:00
4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017-12-16 22:44:00
...,...,...,...,...,...,...,...,...
362793,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024-01-31 23:07:00
362794,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024-02-09 14:09:00
362795,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023-11-07 18:34:00
362796,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024-02-11 16:15:00


In [51]:
npd_officers_clean['occurred_year'] = npd_officers_clean['Incident Occurred Datetime'].dt.year
npd_officers_clean['occurred_hour_minutes'] = npd_officers_clean['Incident Occurred Datetime'].dt.strftime('%H:%M')

In [53]:
npd_officers_clean['reported_year'] = npd_officers_clean['Incident Reported Datetime'].dt.year
npd_officers_clean['reported_hour_minutes'] = npd_officers_clean['Incident Reported Datetime'].dt.strftime('%H:%M')

In [54]:
npd_officers_clean

,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,Incident Reported Datetime,occurred_year,occurred_hour_minutes,reported_year,reported_hour_minutes
0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019-07-01 18:17:00,2019,17:31,2019,18:17
1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018-03-31 19:41:00,2018,12:21,2018,19:41
2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020-04-06 13:11:00,2020,11:15,2020,13:11
3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017-05-29 04:27:00,2017,02:33,2017,04:27
4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017-12-16 22:44:00,2017,18:30,2017,22:44
...,...,...,...,...,...,...,...,...,...,...,...,...
362793,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024-01-31 23:07:00,2024,19:00,2024,23:07
362794,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024-02-09 14:09:00,2024,10:17,2024,14:09
362795,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023-11-07 18:34:00,2023,15:58,2023,18:34
362796,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024-02-11 16:15:00,2024,14:00,2024,16:15


In [56]:
npd_officers_clean['Time Taken'] = npd_officers_clean['reported_hour_minutes']- npd_officers_clean['occurred_hour_minutes']

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [57]:
npd_officers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362797 entries, 0 to 362797
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   index                       362797 non-null  int64         
 1   Report Type Description     362797 non-null  object        
 2   Investigation Status        362797 non-null  object        
 3   Incident Occurred           362797 non-null  string        
 4   Incident Reported           362797 non-null  string        
 5   ZIP Code                    362043 non-null  float64       
 6   Incident Occurred Datetime  362797 non-null  datetime64[ns]
 7   Incident Reported Datetime  362797 non-null  datetime64[ns]
 8   occurred_year               362797 non-null  int32         
 9   occurred_hour_minutes       362797 non-null  object        
 10  reported_year               362797 non-null  int32         
 11  reported_hour_minutes       362797 non-null 

In [60]:
npd_officers_clean['Occurred Time'] = pd.to_datetime(npd_officers_clean['occurred_hour_minutes'])

C:\Users\kesha\AppData\Local\Temp\ipykernel_5064\2071434028.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  npd_officers_clean['Occurred Time'] = pd.to_datetime(npd_officers_clean['occurred_hour_minutes'])


In [61]:
npd_officers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362797 entries, 0 to 362797
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   index                       362797 non-null  int64         
 1   Report Type Description     362797 non-null  object        
 2   Investigation Status        362797 non-null  object        
 3   Incident Occurred           362797 non-null  string        
 4   Incident Reported           362797 non-null  string        
 5   ZIP Code                    362043 non-null  float64       
 6   Incident Occurred Datetime  362797 non-null  datetime64[ns]
 7   Incident Reported Datetime  362797 non-null  datetime64[ns]
 8   occurred_year               362797 non-null  int32         
 9   occurred_hour_minutes       362797 non-null  object        
 10  reported_year               362797 non-null  int32         
 11  reported_hour_minutes       362797 non-null 

In [62]:
npd_officers_clean['Reported Time'] = pd.to_datetime(npd_officers_clean['reported_hour_minutes'])

C:\Users\kesha\AppData\Local\Temp\ipykernel_5064\1434245554.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  npd_officers_clean['Reported Time'] = pd.to_datetime(npd_officers_clean['reported_hour_minutes'])


In [66]:
npd_officers_clean['Time Taken'] = npd_officers_clean['Reported Time']- npd_officers_clean['Occurred Time']

In [68]:
npd_officers_clean

,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,Incident Reported Datetime,occurred_year,occurred_hour_minutes,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken
0,0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0,2019-07-01 17:31:00,2019-07-01 18:17:00,2019,17:31,2019,18:17,2024-02-24 17:31:00,2024-02-24 18:17:00,0 days 00:46:00
1,1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0,2018-03-31 12:21:00,2018-03-31 19:41:00,2018,12:21,2018,19:41,2024-02-24 12:21:00,2024-02-24 19:41:00,0 days 07:20:00
2,2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0,2020-04-06 11:15:00,2020-04-06 13:11:00,2020,11:15,2020,13:11,2024-02-24 11:15:00,2024-02-24 13:11:00,0 days 01:56:00
3,3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0,2017-05-29 02:33:00,2017-05-29 04:27:00,2017,02:33,2017,04:27,2024-02-24 02:33:00,2024-02-24 04:27:00,0 days 01:54:00
4,4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0,2017-12-16 18:30:00,2017-12-16 22:44:00,2017,18:30,2017,22:44,2024-02-24 18:30:00,2024-02-24 22:44:00,0 days 04:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362793,865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0,2024-01-31 19:00:00,2024-01-31 23:07:00,2024,19:00,2024,23:07,2024-02-24 19:00:00,2024-02-24 23:07:00,0 days 04:07:00
362794,865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0,2024-02-09 10:17:00,2024-02-09 14:09:00,2024,10:17,2024,14:09,2024-02-24 10:17:00,2024-02-24 14:09:00,0 days 03:52:00
362795,865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0,2023-11-07 15:58:00,2023-11-07 18:34:00,2023,15:58,2023,18:34,2024-02-24 15:58:00,2024-02-24 18:34:00,0 days 02:36:00
362796,865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0,2024-02-11 14:00:00,2024-02-11 16:15:00,2024,14:00,2024,16:15,2024-02-24 14:00:00,2024-02-24 16:15:00,0 days 02:15:00


In [70]:
npd_officers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362797 entries, 0 to 362797
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype          
---  ------                      --------------   -----          
 0   index                       362797 non-null  int64          
 1   Report Type Description     362797 non-null  object         
 2   Investigation Status        362797 non-null  object         
 3   Incident Occurred           362797 non-null  string         
 4   Incident Reported           362797 non-null  string         
 5   ZIP Code                    362043 non-null  float64        
 6   Incident Occurred Datetime  362797 non-null  datetime64[ns] 
 7   Incident Reported Datetime  362797 non-null  datetime64[ns] 
 8   occurred_year               362797 non-null  int32          
 9   occurred_hour_minutes       362797 non-null  object         
 10  reported_year               362797 non-null  int32          
 11  reported_hour_minutes       362

In [81]:
npd_officers_clean['Time Taken'] = npd_officers_clean['Time Taken'].dt.strftime('%H:%M')

AttributeError: 'TimedeltaProperties' object has no attribute 'strftime'

In [82]:
mnpd_officers_clean = npd_officers_clean.loc[npd_officers_clean['reported_year'] == ("2023")

SyntaxError: incomplete input (1585075015.py, line 1)

In [83]:
#filtered the year for 2023 to match the year of the data in my other notebooks. Seen there were approximately 53,000 dispatched, and closed investigations for Nashville in 2023
npd_officers_clean.loc[npd_officers_clean['reported_year'] == 2023]

,index,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code,Incident Occurred Datetime,Incident Reported Datetime,occurred_year,occurred_hour_minutes,reported_year,reported_hour_minutes,Occurred Time,Reported Time,Time Taken
83,145,DISPATCHED,Closed,6/1/2023 18:30,6/1/2023 19:40,37072.0,2023-06-01 18:30:00,2023-06-01 19:40:00,2023,18:30,2023,19:40,2024-02-24 18:30:00,2024-02-24 19:40:00,0 days 01:10:00
1523,3540,DISPATCHED,Closed,5/30/2023 17:00,6/2/2023 16:51,37217.0,2023-05-30 17:00:00,2023-06-02 16:51:00,2023,17:00,2023,16:51,2024-02-24 17:00:00,2024-02-24 16:51:00,-1 days +23:51:00
2134,4867,DISPATCHED,Closed,6/3/2023 5:30,6/3/2023 7:56,37219.0,2023-06-03 05:30:00,2023-06-03 07:56:00,2023,05:30,2023,07:56,2024-02-24 05:30:00,2024-02-24 07:56:00,0 days 02:26:00
2188,5020,DISPATCHED,Closed,6/4/2023 15:58,6/4/2023 15:58,37216.0,2023-06-04 15:58:00,2023-06-04 15:58:00,2023,15:58,2023,15:58,2024-02-24 15:58:00,2024-02-24 15:58:00,0 days 00:00:00
2270,5226,DISPATCHED,Closed,6/1/2023 10:06,6/2/2023 10:06,37216.0,2023-06-01 10:06:00,2023-06-02 10:06:00,2023,10:06,2023,10:06,2024-02-24 10:06:00,2024-02-24 10:06:00,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362784,865137,DISPATCHED,Closed,9/24/2023 17:00,9/28/2023 18:10,37013.0,2023-09-24 17:00:00,2023-09-28 18:10:00,2023,17:00,2023,18:10,2024-02-24 17:00:00,2024-02-24 18:10:00,0 days 01:10:00
362788,865148,DISPATCHED,Closed,8/12/2023 13:00,8/12/2023 14:15,37214.0,2023-08-12 13:00:00,2023-08-12 14:15:00,2023,13:00,2023,14:15,2024-02-24 13:00:00,2024-02-24 14:15:00,0 days 01:15:00
362790,865156,DISPATCHED,Closed,9/14/2023 19:00,9/14/2023 21:28,37208.0,2023-09-14 19:00:00,2023-09-14 21:28:00,2023,19:00,2023,21:28,2024-02-24 19:00:00,2024-02-24 21:28:00,0 days 02:28:00
362792,865158,DISPATCHED,Closed,11/2/2023 7:30,11/2/2023 9:05,37203.0,2023-11-02 07:30:00,2023-11-02 09:05:00,2023,07:30,2023,09:05,2024-02-24 07:30:00,2024-02-24 09:05:00,0 days 01:35:00


In [84]:
npd_officers_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362797 entries, 0 to 362797
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype          
---  ------                      --------------   -----          
 0   index                       362797 non-null  int64          
 1   Report Type Description     362797 non-null  object         
 2   Investigation Status        362797 non-null  object         
 3   Incident Occurred           362797 non-null  string         
 4   Incident Reported           362797 non-null  string         
 5   ZIP Code                    362043 non-null  float64        
 6   Incident Occurred Datetime  362797 non-null  datetime64[ns] 
 7   Incident Reported Datetime  362797 non-null  datetime64[ns] 
 8   occurred_year               362797 non-null  int32          
 9   occurred_hour_minutes       362797 non-null  object         
 10  reported_year               362797 non-null  int32          
 11  reported_hour_minutes       362

In [89]:
#seeing each individual zip code
npd_officers_clean['ZIP Code'].unique()

array([37214., 37013., 37207., 37205., 37076., 37201., 37208., 37211.,
       37216., 37203., 37228., 37209., 37220., 37072., 37218., 37206.,
       37115., 37212., 37221., 37217., 37138., 37210., 37189., 37213.,
       37219., 37204., 37215., 37240., 37080., 37027.,    nan, 37167.,
       37049., 37135., 37015., 37122., 37086., 37143., 37064., 37232.,
       37130., 37066.,  3701., 19153., 37330., 27707., 91107., 27216.,
       89502., 29501., 37082., 37075., 37043., 37129., 17202., 39202.,
       37087., 37185., 30139., 38562., 37179., 38109., 30318., 37028.,
       37078., 37090., 60123., 37274., 37067., 38401.])

In [94]:
time_by_zip = npd_officers_clean.groupby('ZIP Code')['Time Taken'].mean()
time_by_zip

ZIP Code
3701.0    0 days 14:12:00
17202.0   0 days 09:18:00
19153.0   0 days 07:25:00
27216.0   0 days 01:03:00
27707.0   0 days 04:59:00
                ...      
38562.0   0 days 01:19:00
39202.0   0 days 04:17:00
60123.0   0 days 10:31:00
89502.0   0 days 03:01:00
91107.0   0 days 00:00:00
Name: Time Taken, Length: 69, dtype: timedelta64[ns]

In [95]:
pd.set_option("display.max_rows", None)
print(time_by_zip)

ZIP Code
3701.0                0 days 14:12:00
17202.0               0 days 09:18:00
19153.0               0 days 07:25:00
27216.0               0 days 01:03:00
27707.0               0 days 04:59:00
29501.0               0 days 05:55:00
30139.0             -1 days +15:21:00
30318.0               0 days 12:45:00
37013.0   -1 days +23:52:45.166740040
37015.0   -1 days +23:58:56.052631579
37027.0     0 days 00:12:07.317073170
37028.0               0 days 00:29:00
37043.0               0 days 04:57:00
37049.0               0 days 00:38:00
37064.0             -1 days +17:24:40
37066.0               0 days 08:22:00
37067.0               0 days 04:21:00
37072.0     0 days 00:13:31.325301204
37075.0               0 days 02:44:20
37076.0   -1 days +23:53:17.075981275
37078.0               0 days 01:06:00
37080.0     0 days 00:05:11.512770137
37082.0               0 days 00:00:00
37086.0     0 days 00:33:12.631578947
37087.0             -1 days +23:22:00
37090.0               0 days 04:19:00
371

In [99]:
# getting the average calls per zipcode 
calls_per_zip = npd_officers_clean['ZIP Code'].value_counts()
calls_per_zip

ZIP Code
37207.0    37231
37013.0    36344
37211.0    34391
37115.0    32093
37208.0    20804
37203.0    20714
37206.0    18955
37217.0    18864
37210.0    18468
37209.0    17808
37076.0    16662
37214.0    15993
37218.0    10013
37221.0     8467
37201.0     8142
37216.0     7829
37138.0     5224
37212.0     5130
37204.0     4456
37205.0     4179
37219.0     3516
37215.0     3246
37072.0     2241
37228.0     2169
37189.0     2144
37213.0     1966
37027.0     1804
37080.0     1527
37220.0     1028
37135.0      214
37015.0      152
37240.0      139
37122.0       49
37086.0       19
37143.0       16
37043.0        3
37064.0        3
37167.0        3
37075.0        3
37087.0        3
37232.0        2
37130.0        2
38109.0        1
30139.0        1
38562.0        1
37179.0        1
37090.0        1
30318.0        1
37028.0        1
37078.0        1
60123.0        1
37274.0        1
37067.0        1
37185.0        1
37049.0        1
39202.0        1
17202.0        1
37129.0        1
37082